In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import scipy.io
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
df_train = pd.read_csv("../input/iris-dl-encodings/Train_rin.csv")
df_test = pd.read_csv("../input/iris-dl-encodings/Valid_rin.csv")

In [4]:
X_train = pd.DataFrame(np.stack(df_train['encoding'].apply(lambda x: np.fromstring( x.replace('\n','').replace('[','').replace(']','').replace('  ',' '), sep=' '))))
X_test = pd.DataFrame(np.stack(df_test['encoding'].apply(lambda x: np.fromstring( x.replace('\n','').replace('[','').replace(']','').replace('  ',' '), sep=' '))))
y_train = LabelEncoder().fit_transform(df_train['label'])
y_test = LabelEncoder().fit_transform(df_test['label'])

In [5]:
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import lightgbm as lgbm
import catboost as cb

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [6]:
SEED = 123
cross_valid_scores = {}
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score, make_scorer

Logistic regression

In [9]:
%%time
parameters = {     "C": [ 0.01,0.1, 1.,10, 50],

    "penalty": [ "l1","l2"]
}
SEED = 123
model_logistic_regression = LogisticRegression(
    random_state=SEED,
    class_weight="balanced",
    solver="liblinear",
    
)

model_logistic_regression = GridSearchCV(
    model_logistic_regression, 
    parameters, 
    cv=StratifiedKFold(n_splits=2),
    scoring= 'roc_auc_ovr',
    verbose = 10,
)

model_logistic_regression.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_logistic_regression.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_logistic_regression.best_score_:.3f}'
)
cross_valid_scores['logistic_regression'] = model_logistic_regression.best_score_
print('-----')


Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV 1/2; 1/10] START C=0.01, penalty=l1.........................................
[CV 1/2; 1/10] END .......................C=0.01, penalty=l1; total time=   0.6s
[CV 2/2; 1/10] START C=0.01, penalty=l1.........................................
[CV 2/2; 1/10] END .......................C=0.01, penalty=l1; total time=   0.6s
[CV 1/2; 2/10] START C=0.01, penalty=l2.........................................
[CV 1/2; 2/10] END .......................C=0.01, penalty=l2; total time=   1.0s
[CV 2/2; 2/10] START C=0.01, penalty=l2.........................................
[CV 2/2; 2/10] END .......................C=0.01, penalty=l2; total time=   1.1s
[CV 1/2; 3/10] START C=0.1, penalty=l1..........................................
[CV 1/2; 3/10] END ........................C=0.1, penalty=l1; total time=   0.6s
[CV 2/2; 3/10] START C=0.1, penalty=l1..........................................
[CV 2/2; 3/10] END ........................C=0.1

In [10]:
res = pd.concat([pd.DataFrame(model_logistic_regression.cv_results_["params"]),pd.DataFrame(model_logistic_regression.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_logistic_regression.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_logistic_regression.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res


0.9648692810457516
0.9992541984843796


,C,penalty,AUC_ROC
0,0.01,l1,0.500000
1,0.01,l2,0.992512
2,0.10,l1,0.500000
3,0.10,l2,0.992594
4,1.00,l1,0.500000
5,1.00,l2,0.992691
6,10.00,l1,0.992791
7,10.00,l2,0.993715
8,50.00,l1,0.993463
9,50.00,l2,0.994621


Decision Tree Classifier


In [11]:
%%time



parameters = {
    "max_depth": [3, 5, 21,51,101,181,300,450, 600],
}

model_desicion_tree = DecisionTreeClassifier(
    random_state=SEED,
    class_weight='balanced',
)

model_desicion_tree = GridSearchCV(
    model_desicion_tree, 
    parameters, 
    cv=StratifiedKFold(n_splits=2),
    scoring='f1_macro',
    verbose = 10,
)

model_desicion_tree.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_desicion_tree.best_params_}')
print(
    f'Mean cross-validated  score of the best_estimator: ' + \
    f'{model_desicion_tree.best_score_:.3f}'
)
cross_valid_scores['desicion_tree'] = model_desicion_tree.best_score_
print('-----')

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV 1/2; 1/9] START max_depth=3.................................................
[CV 1/2; 1/9] END ...............................max_depth=3; total time=   0.1s
[CV 2/2; 1/9] START max_depth=3.................................................
[CV 2/2; 1/9] END ...............................max_depth=3; total time=   0.1s
[CV 1/2; 2/9] START max_depth=5.................................................
[CV 1/2; 2/9] END ...............................max_depth=5; total time=   0.2s
[CV 2/2; 2/9] START max_depth=5.................................................
[CV 2/2; 2/9] END ...............................max_depth=5; total time=   0.2s
[CV 1/2; 3/9] START max_depth=21................................................
[CV 1/2; 3/9] END ..............................max_depth=21; total time=   0.9s
[CV 2/2; 3/9] START max_depth=21................................................
[CV 2/2; 3/9] END ..............................m

In [12]:

res = pd.concat([pd.DataFrame(model_desicion_tree.cv_results_["params"]),pd.DataFrame(model_desicion_tree.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_desicion_tree.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_desicion_tree.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res

0.7818627450980392
0.8896939310521148


,max_depth,AUC_ROC
0,3,0.007432
1,5,0.010408
2,21,0.041198
3,51,0.131322
4,101,0.411082
5,181,0.421530
6,300,0.421530
7,450,0.421530
8,600,0.421530


Random Forest Classifier

In [26]:

%%time
parameters = {
    "n_estimators": [ 1800,2000, 2500], 
    "max_depth": [  9, 11,13],
}

model_random_forest = RandomForestClassifier(
    random_state=SEED,
    class_weight='balanced',
)

model_random_forest = GridSearchCV(
    model_random_forest, 
    parameters, 
    cv=StratifiedKFold(n_splits=2),
    scoring='roc_auc_ovr',
    verbose = 10,
)

model_random_forest.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_random_forest.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: '+ \
    f'{model_random_forest.best_score_:.3f}'
)
cross_valid_scores['random_forest'] = model_random_forest.best_score_
print('-----')

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV 1/2; 1/9] START max_depth=9, n_estimators=1800..............................
[CV 1/2; 1/9] END ............max_depth=9, n_estimators=1800; total time=  39.8s
[CV 2/2; 1/9] START max_depth=9, n_estimators=1800..............................
[CV 2/2; 1/9] END ............max_depth=9, n_estimators=1800; total time=  39.9s
[CV 1/2; 2/9] START max_depth=9, n_estimators=2000..............................
[CV 1/2; 2/9] END ............max_depth=9, n_estimators=2000; total time=  44.3s
[CV 2/2; 2/9] START max_depth=9, n_estimators=2000..............................
[CV 2/2; 2/9] END ............max_depth=9, n_estimators=2000; total time=  44.4s
[CV 1/2; 3/9] START max_depth=9, n_estimators=2500..............................
[CV 1/2; 3/9] END ............max_depth=9, n_estimators=2500; total time=  55.2s
[CV 2/2; 3/9] START max_depth=9, n_estimators=2500..............................
[CV 2/2; 3/9] END ............max_depth=9, n_esti

In [25]:

res = pd.concat([pd.DataFrame(model_random_forest.cv_results_["params"]),pd.DataFrame(model_random_forest.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_random_forest.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_random_forest.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res

0.9060457516339869
0.998703162155851


,max_depth,n_estimators,AUC_ROC
0,9,500,0.967345
1,9,1000,0.976370
2,9,1800,0.978672
3,11,500,0.968582
4,11,1000,0.976991
5,11,1800,0.980015
6,13,500,0.971653
7,13,1000,0.976306
8,13,1800,0.978243


ADABOOST

In [23]:
%%time
parameters = {
    "n_estimators": [ 10,100, 300], 
    "learning_rate": [ 0.1, 0.5, 1.1, 1.2, 1.3 ],
}

model_adaboost = AdaBoostClassifier(
    random_state=SEED,
)

model_adaboost = GridSearchCV(
    model_adaboost, 
    parameters, 
    cv=StratifiedKFold(n_splits=2,shuffle = True, random_state= SEED),
    scoring='roc_auc_ovr',
    verbose = 10,
)

model_adaboost.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_adaboost.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: '+ \
    f'{model_adaboost.best_score_:.3f}'
)
cross_valid_scores['ada_boost'] = model_adaboost.best_score_
print('-----')

Fitting 2 folds for each of 15 candidates, totalling 30 fits
[CV 1/2; 1/15] START learning_rate=0.1, n_estimators=10.........................
[CV 1/2; 1/15] END .......learning_rate=0.1, n_estimators=10; total time=   0.7s
[CV 2/2; 1/15] START learning_rate=0.1, n_estimators=10.........................
[CV 2/2; 1/15] END .......learning_rate=0.1, n_estimators=10; total time=   0.7s
[CV 1/2; 2/15] START learning_rate=0.1, n_estimators=100........................
[CV 1/2; 2/15] END ......learning_rate=0.1, n_estimators=100; total time=   5.5s
[CV 2/2; 2/15] START learning_rate=0.1, n_estimators=100........................
[CV 2/2; 2/15] END ......learning_rate=0.1, n_estimators=100; total time=   5.4s
[CV 1/2; 3/15] START learning_rate=0.1, n_estimators=300........................
[CV 1/2; 3/15] END ......learning_rate=0.1, n_estimators=300; total time=  16.1s
[CV 2/2; 3/15] START learning_rate=0.1, n_estimators=300........................
[CV 2/2; 3/15] END ......learning_rate=0.1, n_es

In [24]:

res = pd.concat([pd.DataFrame(model_adaboost.cv_results_["params"]),pd.DataFrame(model_adaboost.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_adaboost.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_adaboost.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res

0.06781045751633986
0.8356058192973352


,learning_rate,n_estimators,AUC_ROC
0,0.1,10,0.532313
1,0.1,100,0.655156
2,0.1,300,0.654526
3,0.5,10,0.525985
4,0.5,100,0.583125
5,0.5,300,0.583078
6,1.1,10,0.507238
7,1.1,100,0.507889
8,1.1,300,0.508405
9,1.2,10,0.507329


SVC

In [17]:
%%time
parameters = {
    "C": [ 20, 50,100],
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
}

model_svc = SVC(
    random_state=SEED,
    class_weight="balanced",
    probability=True,
)

model_svc = GridSearchCV(
    model_svc, 
    parameters, 
    cv=StratifiedKFold(n_splits=2),
    scoring='roc_auc_ovr',
    verbose = 10,
)

model_svc.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_svc.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_svc.best_score_:.3f}'
)
cross_valid_scores['svc'] = model_svc.best_score_
print('-----')

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV 1/2; 1/12] START C=20, kernel=linear........................................
[CV 1/2; 1/12] END ......................C=20, kernel=linear; total time=   2.9s
[CV 2/2; 1/12] START C=20, kernel=linear........................................
[CV 2/2; 1/12] END ......................C=20, kernel=linear; total time=   2.9s
[CV 1/2; 2/12] START C=20, kernel=poly..........................................
[CV 1/2; 2/12] END ........................C=20, kernel=poly; total time=   3.0s
[CV 2/2; 2/12] START C=20, kernel=poly..........................................
[CV 2/2; 2/12] END ........................C=20, kernel=poly; total time=   3.0s
[CV 1/2; 3/12] START C=20, kernel=rbf...........................................
[CV 1/2; 3/12] END .........................C=20, kernel=rbf; total time=   3.3s
[CV 2/2; 3/12] START C=20, kernel=rbf...........................................
[CV 2/2; 3/12] END .........................C=20

In [18]:
res = pd.concat([pd.DataFrame(model_svc.cv_results_["params"]),pd.DataFrame(model_svc.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_svc.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_svc.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res

0.9681372549019608
0.9987278625082726


,C,kernel,AUC_ROC
0,20,linear,0.921308
1,20,poly,0.859976
2,20,rbf,0.889640
3,20,sigmoid,0.923366
4,50,linear,0.921307
5,50,poly,0.859976
6,50,rbf,0.889640
7,50,sigmoid,0.923376
8,100,linear,0.921307
9,100,poly,0.859976


GaussianNB

In [19]:
%%time
parameters =  {'var_smoothing': np.logspace(0,-9, num=100)}

model_gaussianNB = GaussianNB()

model_gaussianNB = GridSearchCV(
    model_gaussianNB, 
    parameters, 
    cv=StratifiedKFold(n_splits=2),
    scoring='roc_auc_ovr',
    verbose = 10,
    n_jobs = -1
)

model_gaussianNB.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_gaussianNB.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_gaussianNB.best_score_:.3f}'
)
cross_valid_scores['svc'] = model_gaussianNB.best_score_
print('-----')

Fitting 2 folds for each of 100 candidates, totalling 200 fits
-----
Best parameters {'var_smoothing': 0.08111308307896872}
Mean cross-validated accuracy score of the best_estimator: 0.996
-----
CPU times: user 1.28 s, sys: 136 ms, total: 1.42 s
Wall time: 28.3 s


In [20]:
res = pd.concat([pd.DataFrame(model_gaussianNB.cv_results_["params"]),pd.DataFrame(model_gaussianNB.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_gaussianNB.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_gaussianNB.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res

0.9321895424836601
0.9994244353398776


,var_smoothing,AUC_ROC
0,1.000000e+00,0.995715
1,8.111308e-01,0.995776
2,6.579332e-01,0.995823
3,5.336699e-01,0.995874
4,4.328761e-01,0.995925
...,...,...
95,2.310130e-09,0.789042
96,1.873817e-09,0.789042
97,1.519911e-09,0.789042
98,1.232847e-09,0.789042


KNeighborsClassifier


In [21]:
%%time
parameters =    {
        'n_neighbors': [3, 5, 7, 9] # usually odd numbers
    }

model_KNN =  KNeighborsClassifier()

model_KNN = GridSearchCV(
    model_KNN, 
    parameters, 
    cv=StratifiedKFold(n_splits=2),
    scoring='roc_auc_ovr',
    verbose = 10,
    n_jobs = -1
 
)

model_KNN.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_KNN.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_KNN.best_score_:.3f}'
)
cross_valid_scores['svc'] = model_KNN.best_score_
print('-----')

res = pd.concat([pd.DataFrame(model_KNN.cv_results_["params"]),pd.DataFrame(model_KNN.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_KNN.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_KNN.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res

Fitting 2 folds for each of 4 candidates, totalling 8 fits
-----
Best parameters {'n_neighbors': 9}
Mean cross-validated accuracy score of the best_estimator: 0.958
-----
0.7001633986928104
0.9915384263675471
CPU times: user 705 ms, sys: 538 ms, total: 1.24 s
Wall time: 1.14 s


,n_neighbors,AUC_ROC
0,3,0.943046
1,5,0.951213
2,7,0.956471
3,9,0.957653
